<a href="https://colab.research.google.com/github/mtlmh34/AML_Project/blob/main/grp%20proj_2.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import dataset, packages

In [2]:
from google.colab import files

In [3]:
files.upload()

Saving fake_job_postings.csv to fake_job_postings.csv


In [11]:
import numpy as np
import pandas as pd
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model, tree, model_selection, ensemble, svm, metrics, naive_bayes
from sklearn.model_selection import RandomizedSearchCV
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, fbeta_score
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [18]:
df=pd.read_csv('fake_job_postings.csv')

### Data preprocessing

In [19]:
# drop columns with more than 50% missing data
# drop column for industry as it has too many unique values

df.drop(columns=['department', 'salary_range', 'industry'], inplace = True)

In [20]:
#-------------------start cleaning for location column

#--------1. split the columns to country, state, and city
df['country']=df.location.str.split(',',expand=True)[:][0]
df['state']=df.location.str.split(',',expand=True)[:][1]
df['city']=df.location.str.split(',',expand=True)[:][2]

#----------2. replacing different kinds of missing value to np.nan
df.state = df.state.str.strip()
df.state.fillna(value=np.nan, inplace=True)
df.state.replace('', np.nan, inplace=True)
df.state.replace(' ', np.nan, inplace=True)

#----------3. clear the whitespaces and signs at the start/end
df.city = df.city.str.strip(' /:\\')

#---------4. replacing different kinds of missing value to np.nan
df.city.fillna(value=np.nan, inplace=True)
df.city.replace('', np.nan, inplace=True)
df.city.replace(' ', np.nan, inplace=True)
df['city']=df.city.str.lower()

#---------

# drop unnecessary columns
df.drop(['location'], axis = 1, inplace = True)

# too many unique entries in city, hence dropped column for city
df.drop(columns=['city'], axis=1, inplace = True)

In [21]:
#-------------------start cleaning for text columns

#extract text dataframe
text_df = df[["title", "company_profile", "description", "requirements", "benefits"]]
text_df = text_df.fillna(' ')


#preprocessing of text_df, concat. all columns
text = text_df[text_df.columns[0:-1]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)

In [22]:
#cleaning text
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)                    #links
    text = re.sub('<.*?>+', '', text)                                   #punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)     #punctuations
    text = re.sub('\n', '', text)                                       #remove \n
    text = re.sub('\w*\d\w*', '', text)                                 #remove numbers in between words
    return text

# Apply the cleaning function to both test and training datasets
text = text.apply(lambda x: clean_text(x))

In [23]:
# remove stop words

stop_words = stopwords.words('english')

def remove_stopwords(text):
  words = [w for w in text if w not in stop_words]
  return words

def combine_text(list_of_text):
  combined_text = ' '.join(list_of_text)
  return combined_text

tokenizer = RegexpTokenizer(r'\w+')                       #tokenize
text = text.apply(lambda x: tokenizer.tokenize(x))
text = text.apply(lambda x : remove_stopwords(x))         #remove stopwords
text = text.apply(lambda x : combine_text(x))             #combine elements of words

In [24]:
# Identify which text feature is not populated before dropping these text columns.

columns_to_remove = ["company_profile", "description", "requirements", "benefits", "title"]


# create new binary columns to check if attribute field is populated in the record.
# note that all records have 'title' information, so no additional binary column is required for title. 

for i in columns_to_remove:
  df[i] = df[i].fillna("unspecified")
new_columns = ["has_company_profile", "has_description", "has_requirements", "has_benefits"]

for i in range(len(new_columns)):
  df[new_columns[i]] = df[columns_to_remove[i]].apply(lambda x: 0 if x == "unspecified" else 1)

df.drop(columns = columns_to_remove, axis = 1, inplace = True)

In [25]:
# column for job_id is not required and hence, dropped

df.drop(columns = ['job_id'], axis = 1, inplace = True)

In [26]:
#-------------------cleaning categorical columns 

category_columns = ['employment_type', 'required_experience', 'required_education', 
                    'function', 'country', 'state']

# when populating missing fields with unspecified, the attribute name is added as a 
# prefix for unspecified to preserve the attribute information. this is required as 
# all categorical columns will be converted into indicator variables in the later step.

for i in category_columns:
  df[i] = df[i].fillna(i + "_unspecified")


# country codes duplicated in country and state columns

duplicated = ['BE', 'DE', 'PA', 'NL', 'ID', 'PL', 'IL', 'IN', 'RS', 'ES', 'SA', 
              'SD', 'MT', 'MA', 'IS', 'SK', 'HR', 'BR', 'TN', 'JM', 'CO', 'CN', 
              'AL', 'AR', 'CH', 'CA', 'BY']


# remove duplicated codes from state column

state_features = pd.get_dummies(df['state'])
state_features.drop(columns = duplicated, axis = 1, inplace = True)


# convert all categorical columns into indicator variables and concate them together.

categorical_features = pd.concat([pd.get_dummies(df['employment_type']), 
                                  pd.get_dummies(df['required_experience']), 
                                  pd.get_dummies(df['required_education']),
                                  pd.get_dummies(df['country']),
                                  state_features],
                                  axis = 1)

df.drop(columns = category_columns, axis = 1, inplace = True)

In [27]:
# train-test-split

# separate target column from dataframe
df_combined = pd.concat([df, categorical_features, text], axis =1, ignore_index=False)
df_combined.rename(columns={ df_combined.columns[-1]: "text" }, inplace = True)
X_data = df_combined.drop('fraudulent', axis = 1)
y_data = df_combined['fraudulent']

X_train_df, X_test_df, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

In [ ]:
# apply vectoriser after train-test-split

# TF_IDF vectorizer
tfidf = TfidfVectorizer(ngram_range = (1,2), min_df= 0.05, max_df= 0.5)

# train set
transformed = tfidf.fit_transform(X_train_df['text'])
df_vec = pd.DataFrame(transformed.todense(),columns=tfidf.get_feature_names())

X_train = X_train_df.drop('text', axis = 1)
X_train = pd.concat([X_train.reset_index(), df_vec.reset_index()], axis = 1)
X_train.drop('index', axis = 1, inplace = True)

# test set
test_text = tfidf.transform(X_test_df['text'])
df_vec_test = pd.DataFrame(test_text.todense(),columns=tfidf.get_feature_names())

X_test = X_test_df.drop('text', axis = 1)
X_test = pd.concat([X_test.reset_index(), df_vec_test.reset_index()], axis = 1)
X_test.drop('index', axis = 1, inplace = True)

In [31]:
# additional standardisation step for artificial neural network

#standard scaler that applies to all text vectors
scaler = StandardScaler()
df_vec_scaled = scaler.fit_transform(df_vec)
df_vec_scaled_test = scaler.transform(df_vec_test)

# adding column names
df_vec_scaled=pd.DataFrame(df_vec_scaled, columns=tfidf.get_feature_names())
df_vec_scaled_test=pd.DataFrame(df_vec_scaled_test, columns=tfidf.get_feature_names())

# train set for artificial neural network
X_train_nn = X_train_df.drop('text', axis = 1)
X_train_nn = pd.concat([X_train_nn.reset_index(), df_vec_scaled.reset_index()], axis = 1)
X_train_nn.drop('index', axis = 1, inplace = True)

# test set for artificial neural network
X_test_nn = X_test_df.drop('text', axis = 1)
X_test_nn = pd.concat([X_test_nn.reset_index(), df_vec_scaled_test.reset_index()], axis = 1)
X_test_nn.drop('index', axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [32]:
# minus 1 for X_train_df.shape[1] due to 'text' column in X_train_df

print("Number of text columns: ", X_train.shape[1] - (X_train_df.shape[1] - 1))
print("Number of binary columns: ", X_train_df.shape[1] - 1)

Number of text columns:  668
Number of binary columns:  424


In [33]:
# same checks for neural network

print("Number of text columns: ", X_train_nn.shape[1] - (X_train_df.shape[1] - 1))
print("Number of binary columns: ", X_train_df.shape[1] - 1)

Number of text columns:  668
Number of binary columns:  424


### Evaluate Sampling Strategy

In [34]:
import warnings
warnings.filterwarnings('ignore')

# create function to evaluate whether SMOTE-TOMEK or SMOTE sampling gives a better fbeta score.
# the size of the sampling strategy is also evaluated.
# this is only a quick evaluation of the sampling type and size, hence a simpler model, decision tree is used.

def sampling_evaluation(type, size):
    if type == 'smote-tomek':
        smt = SMOTETomek(random_state=42, sampling_strategy=size)
    elif type == 'smote':
        smt = SMOTE(random_state=42, sampling_strategy=size)

    X_resample, y_resampled = smt.fit_resample(X_train, y_train)

  # train decision tree model
    dt = DecisionTreeClassifier(random_state=42)
    dt.fit(X_resample, y_resampled)

  # evaluate predictions
    y_pred_test = dt.predict(X_test)
    fbeta = fbeta_score(y_test, y_pred_test, beta = 2.0)
    
    return fbeta

In [35]:
# the over-sampling techniques and the corresponding sampling strategy size is as listed below.
sampling_strat = [('smote-tomek', 0.4), ('smote-tomek', 0.5), ('smote-tomek', 0.6), ('smote', 0.4), ('smote', 0.5), ('smote', 0.6)]

for strat in sampling_strat:
  print(strat, ": ", sampling_evaluation(strat[0], strat[1]))

('smote-tomek', 0.4) :  0.7272727272727273
('smote-tomek', 0.5) :  0.6896551724137931
('smote-tomek', 0.6) :  0.669882100750268
('smote', 0.4) :  0.7249466950959489
('smote', 0.5) :  0.6942949407965555
('smote', 0.6) :  0.669882100750268


In [36]:
# SMOTE with sampling strategy of 0.4 has the best performance 
# hence, this will be used to perform over-sampling 

smt = SMOTE(random_state=42, sampling_strategy=0.4)
X_resampled, y_resampled = smt.fit_resample(X_train, y_train)

In [38]:
# the same sampling is performed for the data to be used for artificial neural network  

smt_nn = SMOTE(random_state=42, sampling_strategy=0.4)
X_resampled_nn, y_resampled_nn = smt_nn.fit_resample(X_train_nn, y_train)

In [39]:
print("For all others:\n", y_resampled.value_counts())
print("")
print("For neural network:\n", y_resampled_nn.value_counts())

For all others:
 0    13619
1     5447
Name: fraudulent, dtype: int64

For neural network:
 0    13619
1     5447
Name: fraudulent, dtype: int64


### Naive Bayes

In [40]:
mnb = naive_bayes.MultinomialNB()
mnb.fit(X_resampled, y_resampled)

MultinomialNB()

In [41]:
def model_results(model, X_values, y_values, train_test):
  y_pred = model.predict(X_values)
  y_pred_prob = model.predict_proba(X_values)[:,1]
      
  precision = metrics.precision_score(y_values, y_pred)
  recall = metrics.recall_score(y_values, y_pred)
  f1 = metrics.f1_score(y_values, y_pred)
  roc_auc = metrics.roc_auc_score(y_values, y_pred_prob)
  fbeta = fbeta_score(y_values, y_pred, beta = 2.0)

  cf_matrix = metrics.confusion_matrix(y_values, y_pred, labels=[1,0])

  evaluation_metrics = ['precision', 'recall', 'f1', 'roc_auc', 'f-beta']
  scores = [precision, recall, f1, roc_auc, fbeta]
  print(pd.DataFrame([scores], columns = evaluation_metrics, index = [train_test]))
  print("")
  print("Confusion matrix: ")
  print(cf_matrix)

In [42]:
model_results(mnb, X_resampled, y_resampled, "Train")

       precision    recall        f1   roc_auc    f-beta
Train   0.804167  0.779512  0.791647  0.944064  0.784321

Confusion matrix: 
[[ 4246  1201]
 [ 1034 12585]]


In [43]:
model_results(mnb, X_test, y_test, "Test")

      precision    recall        f1   roc_auc    f-beta
Test   0.301435  0.696133  0.420701  0.911293  0.551664

Confusion matrix: 
[[ 126   55]
 [ 292 3103]]


In [44]:
# identify top features in naive bayes model

def top_features(mnb, X_resampled):
  # match feature names and respective feature counts in each class
  f = {'Feature': X_resampled.columns, 'fraud': mnb.feature_count_[1,:],
          'not_fraud': mnb.feature_count_[0,:]}
  features = pd.DataFrame(data=f)

  # calculate the P(feature|class) by using feature count in the class divided by respective class count
  # 1 count is added to all feature counts to ensure ratio can be calculated later
  features['P (f|fraud)'] = (features['fraud'] +1) / mnb.class_count_[1]
  features['P (f|not_fraud)'] = (features['not_fraud'] +1) / mnb.class_count_[0]

  # ratio of the P(feature|class) is used to evaluate the predictiveness of the token 
  features['Ratio'] = features['P (f|fraud)'] / features['P (f|not_fraud)']

  # print out the top 10 features for each class
  print('The top 20 most predictive tokens for fraud job postings are:')
  print(features.sort_values(by=['Ratio'], ascending= False).head(20)['Feature'].values)

  print('The top 20 most predictive tokens for not fraud job postings are:')
  print(features.sort_values(by=['Ratio'], ascending= False).tail(20)['Feature'].values)

top_features(mnb, X_resampled)

The top 20 most predictive tokens for fraud job postings are:
['Some High School Coursework' 'BH' 'MY' '16' 'NSW' 'entry' 'AGB' 'DA'
 'MD' 'QA' 'ARM' 'AU' 'administrative' 'per' 'internet' 'needed' 'follow'
 'free' 'call' '14']
The top 20 most predictive tokens for not fraud job postings are:
['FR' 'relevant job' 'LND' 'SG' 'IL' 'HK' 'B' 'OR' 'IN' 'consultants' 'BC'
 'IE' 'BE' 'NL' 'abroad' 'N' 'NZ' 'DE' 'I' 'GR']


In [45]:
# additional evaluation with only text columns

# SMOTE with sampling strategy of 0.4 has the best performance 
# hence, this will be used to perform over-sampling 

smt = SMOTE(random_state=42, sampling_strategy=0.4)
X_resampled_text, y_resampled_text = smt.fit_resample(df_vec, y_train)

# model building
mnb_text = naive_bayes.MultinomialNB()
mnb_text.fit(X_resampled_text, y_resampled_text)

model_results(mnb_text, X_resampled_text, y_resampled_text, "Train")

print('')
model_results(mnb_text, df_vec_test, y_test, "Test")

print('')
top_features(mnb_text, X_resampled_text)

       precision    recall        f1   roc_auc   f-beta
Train   0.847513  0.616303  0.713648  0.935203  0.65187

Confusion matrix: 
[[ 3357  2090]
 [  604 13015]]

      precision    recall        f1   roc_auc    f-beta
Test   0.404412  0.607735  0.485651  0.911033  0.552209

Confusion matrix: 
[[ 110   71]
 [ 162 3233]]

The top 20 most predictive tokens for fraud job postings are:
['entry' 'administrative' 'per' 'internet' 'needed' 'follow' 'free' 'call'
 'home' 'center' 'area' 'desire' 'week' 'access' 'calls' 'recruiting'
 'maximize' 'require' 'able work' 'equipment']
The top 20 most predictive tokens for not fraud job postings are:
['digital' 'recruitment' 'media' 'search' 'founded' 'since'
 'experience preferred' 'secure' 'continually' 'trusted' 'engage' 'fun'
 'teaching' 'website' 'amazing' 'relevant job' 'passionate' 'social media'
 'consultants' 'abroad']


### Gradient Boosting

In [ ]:
gbt = ensemble.GradientBoostingClassifier(random_state=42)
gbt.fit(X_resampled, y_resampled)

GradientBoostingClassifier(random_state=42)

In [ ]:
model_results(gbt, X_resampled, y_resampled, "Train")


       precision    recall        f1   roc_auc    f-beta
Train   0.965127  0.939967  0.952381  0.996295  0.944894

Confusion matrix: 
[[ 5120   327]
 [  185 13434]]


In [ ]:
model_results(gbt, X_test, y_test, "Test")

      precision    recall        f1   roc_auc    f-beta
Test   0.684492  0.707182  0.695652  0.971271  0.702525

Confusion matrix: 
[[ 128   53]
 [  59 3336]]


In [ ]:
gbt = {'Feature': gbt.feature_names_in_, 'Importance': gbt.feature_importances_}
df_gbt = pd.DataFrame(gbt)
df_gbt = df_gbt.sort_values(by = 'Importance', key = abs, ascending = False, ignore_index =True)
df_gbt.head(10)

,Feature,Importance
0,has_company_profile,0.264006
1,has_company_logo,0.075283
2,recruiting,0.047239
3,program,0.038407
4,addition,0.027505
5,maximize,0.022703
6,High School or equivalent,0.020293
7,area,0.017774
8,medium,0.017681
9,exciting,0.016849


In [ ]:
def model_build(base_estimator, parameters):
    
    clf = model_selection.RandomizedSearchCV(
        base_estimator, 
        parameters, 
        scoring = 'recall',
        cv = 5)
    
    clf.fit(X_resampled, y_resampled)
    
    print('best score:', clf.best_score_)
    print('best parameters: ', clf.best_params_)

    optimised = clf.best_estimator_
    return optimised

In [ ]:
gradient_grid = {'n_estimators': [x for x in np.arange(start=100,stop=180,step=20)],
               'learning_rate': [0.001, 0.01, 0.1]}

optimised_gbt = model_build(ensemble.GradientBoostingClassifier(random_state=2022), gradient_grid)

In [ ]:
gbt = {'Feature': optimised_gbt.feature_names_in_, 'Importance': optimised_gbt.feature_importances_}
df_gbt = pd.DataFrame(gbt)
df_gbt = df_gbt.sort_values(by = 'Importance', ascending = False, ignore_index =True)
df_gbt.head(10)

In [ ]:
model_results(optimised_gbt, X_resampled, y_resampled, "Train")
model_results(optimised_gbt, X_test, y_test, "Test")

In [ ]:
y_score = optimised_gbt.predict_proba(X_test)[:, 1]

#calculate precision and recall
precision, recall, thresholds = precision_recall_curve(y_test, y_score)

#create precision recall curve
fig, ax = plt.subplots()
ax.plot(recall, precision, color='purple')

#add axis labels to plot
ax.set_title('Precision-Recall Curve')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')

#display plot
plt.show()

### Support Vector Machine

In [ ]:
clf_svm = svm.SVC(probability=True, random_state=42)
clf_svm.fit(X_resampled, y_resampled)

SVC(probability=True, random_state=42)

In [ ]:
model_results(clf_svm, X_resampled, y_resampled, "Train")


       precision    recall        f1  roc_auc    f-beta
Train   0.987323  0.986598  0.986961  0.99962  0.986743

Confusion matrix: 
[[ 5374    73]
 [   69 13550]]


In [ ]:
model_results(clf_svm, X_test, y_test, "Test")

      precision    recall        f1   roc_auc    f-beta
Test   0.843373  0.773481  0.806916  0.978914  0.786517

Confusion matrix: 
[[ 140   41]
 [  26 3369]]


In [ ]:
from sklearn.model_selection import GridSearchCV
def model_build2(base_estimator, parameters):
    
    clf = model_selection.GridSearchCV(
        base_estimator, 
        parameters,
        cv=5)
    
    clf.fit(X_resample, y_resampled)
    
    print('best score:', clf.best_score_)
    print('best parameters: ', clf.best_params_)

    optimised = clf.best_estimator_
    return optimised

In [ ]:
svm_grid = {'C': [0.1, 1.0, 10]}

optimised_svm = model_build2(svm.SVC(random_state=2022, probability=True), svm_grid)

In [ ]:
model_results(optimised_svm, X_resampled, y_resampled, "Train")
model_results(optimised_svm, X_test, y_test, "Test")

In [ ]:
y_score = optimised_svm.predict_proba(X_test)[:, 1]

#calculate precision and recall
precision, recall, thresholds = precision_recall_curve(y_test, y_score)

#create precision recall curve
fig, ax = plt.subplots()
ax.plot(recall, precision, color='purple')

#add axis labels to plot
ax.set_title('Precision-Recall Curve')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')

#display plot
plt.show()

### Neural Network

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation,Embedding,GlobalAveragePooling1D
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers

In [ ]:
X_resampled_nn.shape

(19066, 1091)

In [ ]:
def create_model(optimizer='adam',dropout_rate=0.2):
    model = Sequential()

    model.add(Dense(128, input_shape=(1091,), activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())

    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())

    model.add(Dense(16, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    # compile the model
    model.compile(loss='binary_crossentropy', 
                  optimizer=optimizer, 
                  metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    
    return model

In [ ]:
model = KerasClassifier(build_fn=create_model, verbose=0, validation_split=0.2)
# define the grid search parameters

batch_size = [8,16,24,32]
epochs = [20,30,40]
optimizer = ['SGD', 'RMSprop', 'Adam']
dropout_rate = [0.3, 0.4, 0.5, 0.6]

param_grid = dict(batch_size=batch_size, 
                  epochs=epochs,
                  optimizer=optimizer,
                  dropout_rate=dropout_rate)

grid = RandomizedSearchCV(estimator=model, 
                          n_iter=10,
                          param_distributions=param_grid, 
                          n_jobs=-1, 
                          cv=10, 
                          scoring ='recall',
                          random_state=42)

grid_result = grid.fit(X_resampled_nn, y_resampled_nn, validation_split=0.4)

#### Evaluating best model of SearchCV

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.813660 using {'optimizer': 'RMSprop', 'epochs': 40, 'dropout_rate': 0.5, 'batch_size': 24}
0.055334 (0.071107) with: {'optimizer': 'SGD', 'epochs': 20, 'dropout_rate': 0.4, 'batch_size': 32}
0.000000 (0.000000) with: {'optimizer': 'RMSprop', 'epochs': 20, 'dropout_rate': 0.5, 'batch_size': 8}
0.771130 (0.119521) with: {'optimizer': 'RMSprop', 'epochs': 20, 'dropout_rate': 0.4, 'batch_size': 24}
0.813660 (0.098767) with: {'optimizer': 'RMSprop', 'epochs': 40, 'dropout_rate': 0.5, 'batch_size': 24}
0.605736 (0.093046) with: {'optimizer': 'Adam', 'epochs': 20, 'dropout_rate': 0.5, 'batch_size': 16}
0.548377 (0.107094) with: {'optimizer': 'SGD', 'epochs': 30, 'dropout_rate': 0.4, 'batch_size': 8}
0.095645 (0.085145) with: {'optimizer': 'SGD', 'epochs': 40, 'dropout_rate': 0.5, 'batch_size': 32}
0.533083 (0.110736) with: {'optimizer': 'Adam', 'epochs': 20, 'dropout_rate': 0.6, 'batch_size': 16}
0.000000 (0.000000) with: {'optimizer': 'SGD', 'epochs': 30, 'dropout_rate': 0.6, 'batch_

In [ ]:
# train the best estimator

best=grid_result.best_estimator_
history = best.fit(X_resampled_nn, y_resampled_nn, verbose=1, validation_split=0.3)

In [ ]:
# evaluation of model

# Prediction of test set
Y_pred = best.predict(X_test)

# classification report, confusion matrix
print(classification_report(y_true=y_test, y_pred=Y_pred))
print(confusion_matrix(y_test,Y_pred,labels=[1,0]))

ln_loss = history.history['loss']
ln_val_loss = history.history['val_loss']
ln_p = history.history['precision']
ln_val_p = history.history['val_precision']
ln_r = history.history['recall']
ln_val_r = history.history['val_recall']

#precision plot
plt.figure(figsize=(20,5))
plt.subplot(131)
plt.plot(ln_p)
plt.plot(ln_val_p)
plt.ylabel('Precision')
plt.xlabel('Epoche')
plt.legend(['Training', 'Validation'], loc='lower right')

#recall plot
plt.subplot(132)
plt.plot(ln_r)
plt.plot(ln_val_r)
plt.ylabel('Recall')
plt.xlabel('Epoche')
plt.legend(['Training', 'Validation'], loc='lower right')

#loss plot
plt.subplot(133)
plt.plot(ln_loss)
plt.plot(ln_val_loss)
plt.ylabel('Loss')
plt.xlabel('Epoche')
plt.legend(['Training', 'Validation'], loc='lower right')

#### Tuning the best CV model

In [ ]:
from keras.callbacks import EarlyStopping

best=grid_result.best_estimator_
#early stopping
#es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=20)
best_tuned_1 = best.fit(X_train, y_train, epochs=100)

In [ ]:
#evaluation of tuned model

#rf_matrixt = metrics.confusion_matrix(y_test, y_pred_test, labels=[1,0])

Y_pred_2 = best.predict(X_test)
print(classification_report(y_true=y_test, y_pred=Y_pred_2))
print(confusion_matrix(y_test,Y_pred_2,labels=[1,0]))

ln_loss = best_tuned_1.history['loss']
ln_val_loss = best_tuned_1.history['val_loss']
ln_p = best_tuned_1.history['precision_1']
ln_val_p = best_tuned_1.history['val_precision_1']
ln_r = best_tuned_1.history['recall_1']
ln_val_r = best_tuned_1.history['val_recall_1']


plt.figure(figsize=(20,5))
plt.subplot(131)
plt.plot(ln_p)
plt.plot(ln_val_p)
plt.ylabel('Precision')
plt.xlabel('Epoche')
plt.legend(['Training', 'Validation'], loc='lower right')

plt.subplot(132)
plt.plot(ln_r)
plt.plot(ln_val_r)
plt.ylabel('Recall')
plt.xlabel('Epoche')
plt.legend(['Training', 'Validation'], loc='lower right')

plt.subplot(133)
plt.plot(ln_loss)
plt.plot(ln_val_loss)
plt.ylabel('Loss')
plt.xlabel('Epoche')
plt.legend(['Training', 'Validation'], loc='lower right')

In [ ]:
#Precision recall curve

from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

best=grid_result.best_estimator_
y_score = best.predict_proba(np.array(X_test))[:, 1]

#calculate precision and recall
precision, recall, thresholds = precision_recall_curve(y_test, y_score)

#create precision recall curve
fig, ax = plt.subplots()

fig.set_size_inches(9,6)
ax.plot(recall, precision, color='purple')

#add axis labels to plot
ax.set_title('Precision-Recall Curve of Best Neural Network Model')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')

#display plot
plt.show()

### End of Project